In [2]:
import geopandas as gpd
import rasterio
import numpy as np
from rasterio.plot import show
from rasterio.warp import reproject, Resampling

# 读取 DEM 并计算坡度
with rasterio.open("../data/raw/HydroSHEDS_DEM_90m.tif") as dem:
    elevation = dem.read(1)
    profile = dem.profile

# 计算坡度
slope_x, slope_y = np.gradient(elevation)
slope = np.sqrt(slope_x**2 + slope_y**2)

# 读取 avg_cloud_thickness.tif 并获取其形状和变换信息
with rasterio.open("../data/processed/avg_cloud_thickness.tif") as src:
    cloud_data = src.read(1)
    cloud_profile = src.profile

# 创建一个与 cloud_data 相同大小的数组来存储重采样后的 slope 数据
slope_resampled = np.empty_like(cloud_data)

# 重采样 slope 数据到与 cloud_data 相同的大小
reproject(
    source=slope,
    destination=slope_resampled,
    src_transform=profile['transform'],
    src_crs=profile['crs'],
    dst_transform=cloud_profile['transform'],
    dst_crs=cloud_profile['crs'],
    resampling=Resampling.bilinear
)

# 保存重采样后的坡度特征数据
np.save("../data/processed/slope_resampled.npy", slope_resampled)

# 保存 avg_cloud_thickness 数据为 .npy 文件
np.save("../data/processed/avg_cloud_thickness.npy", cloud_data)